# Instructions

Read the assignment instructions here: https://github.com/markriedl/coinrun-game-ai-assignment

Those new to pytorch may find this [primer](https://colab.research.google.com/drive/1DgkVmi6GksWOByhYVQpyUB4Rk3PUq0Cp) helpful.

Those new to convolutional neural networks may find this [example](https://colab.research.google.com/drive/1740OwEI4oi5QlPtq4UFd3ha44yxef7gU) helpful.

# Installation

Run the following cells

In [1]:
import os
# del os.environ['LD_PRELOAD']
# !apt-get remove libtcmalloc*

In [ ]:
!apt-get update
!apt-get install mpich build-essential qt5-default pkg-config

In [ ]:
!git clone https://github.com/markriedl/coinrun-game-ai-assignment.git

In [ ]:
!pip install -r coinrun-game-ai-assignment/requirements.txt

In [5]:
import sys
sys.path.insert(0, 'coinrun-game-ai-assignment')

In [ ]:
import torch
torch.cuda.is_available()

If the prior cell repots ```False```, then use notebooks settings to turn GPU support on.

In [ ]:
# THIS TESTS THE COINRUN INSTALLATION
from coinrun.random_agent import random_agent

random_agent(max_steps=10)

# Setup

In [8]:
def in_ipynb():
  try:
    result = get_ipython().__class__.__name__
    if 'Shell' in result:
      return True
    else:
      return False
  except:
    return False

IN_PYNB = in_ipynb()



In [9]:
import gym
import os
import math
import random
import numpy as np
from collections import namedtuple
from itertools import count
from PIL import Image

from coinrun import setup_utils, make
import coinrun.main_utils as utils
from coinrun.config import Config
if not IN_PYNB:
    from gym.envs.classic_control import rendering
from coinrun import policies, wrappers

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import pdb

# import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

import PIL
from PIL import Image
import shutil

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Resize the screen to this
RESIZE_CONST = 40 


# Game seed information
NUM_LEVELS = 1 # repeat the same level over and over
EASY_LEVEL = 1 # Start on a very small map, no enemies
EASY_LEVEL2 = 5 # Very small map, no enemies
MEDIUM_LEVEL = 20 # Medium length, no enemies
MEDIUM_LEVEL2 = 45 # Medium length, no enemies
ONE_MONSTER = 10 # Short map with one monster
HARD_LEVEL = 7 # Longer and with monsters
LAVA_LEVEL = 3 # Longer and with lava and pits

# Defaults
RENDER_SCREEN = False
SAVE_FILENAME = 'saved.model'
LOAD_FILENAME = 'saved.model'
MODEL_PATH = 'saved_models' 
SEED = EASY_LEVEL

# Don't play with this
EVAL_EPSILON = 0.1
EVAL_WINDOW_SIZE = 3
EVAL_COUNT = 10
TIMEOUT = 1000
COIN_REWARD = 100

### Data structure for holding experiences for replay
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

### Function for resizing the screen
resize = T.Compose([T.ToPILImage(),
                    T.Resize(RESIZE_CONST, interpolation=Image.CUBIC),
                    T.ToTensor()])

### Save the model. Extra information can be added to the end of the filename
def save_model(model, filename, extras = None):
    if extras is not None:
        filename = filename + '.' + str(extras)
    print("Saving", filename, "...")
    torch.save(model, os.path.join(MODEL_PATH, filename))
    print("Done saving.")
 
### Load the model. If there are multiple versions with extra information at the
### end of the filename, get the latest.
def load_model(filename, extras = None):
    if extras is not None:
        filename = filename + '.' + str(extras)
    model = None
    candidates = [os.path.join(MODEL_PATH, f) for f in os.listdir(MODEL_PATH) if filename in f]
    if len(candidates) > 0:
        candidates = sorted(candidates, key=lambda f:os.stat(f).st_mtime, reverse=True)
        filename = candidates[0]
        print("Loading", filename, "...")
        model = torch.load(filename)
        print("Done loading.")
    return model
  
### Give a text description of the outcome of an episode and also a score
### Score is duration, unless the agent died.
def episode_status(duration, reward):
    status = ""
    score = 0
    if duration >= TIMEOUT:
        status = "timeout"
        score = duration
    elif reward < COIN_REWARD:
        status = "died"
        score = TIMEOUT
    else:
        status = "coin"
        score = duration
    return status, score

Every time the code runs an ```evaluate()``` of the agent's model, a new directory will be created in ```/content/cache``` and screenshots of the agent in action will be stored. Filenames are of the form ```/content/cache/eval_number/eval + screen_number + .jpeg```.

In [79]:
RUN_NUM = 0
SCREEN_SAVE = False
SCREEN_COUNT = 0
SCREEN_SAVE_PREFIX = 'eval'
SCREEN_SAVE_POSTFIX = '.jpeg'
TEMP_DIR = 'cache'
SCREEN_SAVE_RATIO = 0.5

#Screen is numpy array
def save_screen(screen):
  global SCREEN_COUNT
  screen_array_t = np.transpose(screen, (1, 2, 0))
  img = Image.fromarray(np.uint8(screen_array_t * 255))
  width, height = img.size
  img = img.resize((int(width*SCREEN_SAVE_RATIO), int(height*SCREEN_SAVE_RATIO)), Image.LANCZOS)
  if not os.path.isdir(TEMP_DIR):
    os.mkdir(TEMP_DIR)
  if not os.path.isdir(os.path.join(TEMP_DIR, str(RUN_NUM))):
    os.mkdir(os.path.join(TEMP_DIR, str(RUN_NUM)))
  img.save(os.path.join(TEMP_DIR, str(RUN_NUM), SCREEN_SAVE_PREFIX + str(SCREEN_COUNT) + SCREEN_SAVE_POSTFIX), "JPEG")
  SCREEN_COUNT = SCREEN_COUNT + 1

def show_movie(num):
  dir_name = str(num)
  files = []
  for file in os.listdir(os.path.join(TEMP_DIR, dir_name)):
    files.append(file)
  sorted_files = sorted(files, key=lambda f:int(f[len(SCREEN_SAVE_PREFIX):-len(SCREEN_SAVE_POSTFIX)]))
  for file in sorted_files:
    img = Image.open(os.path.join(TEMP_DIR, dir_name, file), 'r')
    plt.imshow(img)
    ipythondisplay.clear_output(wait=True)
    ipythondisplay.display(plt.gcf())

def make_anim(num):
  dir_name = str(num)
  files = []
  images = []
  for file in os.listdir(os.path.join(TEMP_DIR, dir_name)):
    files.append(file)
  sorted_files = sorted(files, key=lambda f:int(f[len(SCREEN_SAVE_PREFIX):-len(SCREEN_SAVE_POSTFIX)]))
  for file in sorted_files:
    try:
      img = Image.open(os.path.join(TEMP_DIR, dir_name, file))
      images.append(img)
    except:
      print(os.path.join(TEMP_DIR, dir_name, file), "did not load.")
  images[0].save('movie' + str(num) + '.gif', "GIF",
                      save_all=True,
                      append_images=images[1:],
                      duration=1)


# Globals

In [107]:

BATCH_SIZE = 128            # How many replay experiences to run through neural net at once
GAMMA = 0.999               # How much to discount the future [0..1]
BOOTSTRAP = 20000           # How many steps to run to fill up replay memory before training starts
TARGET_UPDATE = 0           # Delays updating the network for loss calculations. 0=don't delay, or 1+ number of episodes
REPLAY_CAPACITY = 10000     # How big is the replay memory
EPSILON = 0.9               # Use random action if less than epsilon [0..1]
EVAL_INTERVAL = 10          # How many episodes of training before evaluation
NUM_EPISODES = 200          # Max number of training episodes
RANDOM_SEED = None          # Seed for random number generator, for reproducability, use None for random seed
EPSILON_DECAY_NUM = 1

# Reference Functions

## Unit Testing Functions

In [82]:

def testReplayMemory():
    print("Testing ReplayMemory...")
    capacity = 100
    test_replay_memory = ReplayMemory(capacity)
    for i in range(capacity):
        test_replay_memory.push(i, i, i, i)
    assert (len(test_replay_memory) == capacity),"size test failed"
    for i in range(len(test_replay_memory)):
        item = test_replay_memory.memory[i]
        assert (item[0] == i), "item" + str(i) + "not holding the correct value"
    for i in range(capacity//2):
        test_replay_memory.push(capacity+i, capacity+i, capacity+i, capacity+i)
    assert (len(test_replay_memory) == capacity), "size test 2 failed"
    # check items
    for i in range(len(test_replay_memory)):
        item = test_replay_memory.memory[i]
        if i < capacity // 2:
            assert (item[0] == i+capacity), "not holding the correct value after looping (first half)"
        else:
            assert (item[0] == i), "not holding the correct value after looping (second half)"
    print("ReplayMemory test passed.")
    return True
  
def testMakeBatch():
    print("Testing doMakeBatch...")
    batch_size = 128
    capacity = batch_size * 2
    test_replay_memory = ReplayMemory(capacity)
    state = None
    new_state = None
    action = None
    reward = None
    # Test types and shapes of return values
    for i in range(capacity):
        state = torch.randn(1, 3, 80, 80, device=DEVICE)
        new_state = torch.randn(1, 3, 80, 80, device=DEVICE)
        action = torch.randn(1, 1, device=DEVICE)
        reward = torch.randn(1, 1, device=DEVICE)
        test_replay_memory.push(state, action, new_state, reward)
    states_batch, actions_batch, next_states_batch, rewards_batch, non_final_mask = doMakeBatch(test_replay_memory, batch_size)
    assert(type(states_batch) == torch.Tensor and states_batch.size() == (batch_size, 3, 80, 80)), "states batch not correct shape."
    assert(type(actions_batch) == torch.Tensor and actions_batch.size() == (batch_size, 1)), "actions batch not correct shape."
    assert(type(next_states_batch) == torch.Tensor and next_states_batch.size() == (batch_size, 3, 80, 80)), "next states batch not correct shape."
    assert(type(rewards_batch) == torch.Tensor and rewards_batch.size() == (batch_size, 1)), "rewards batch not correct shape."
    assert(type(non_final_mask) == type(torch.tensor(batch_size, dtype=torch.bool, device=DEVICE)) and non_final_mask.size()[0] == batch_size), "non-final mask not correct shape."

    # Test mask
    test_replay_memory = ReplayMemory(batch_size)
    for i in range(batch_size):
        state = torch.randn(1, 3, 80, 80, device=DEVICE)
        new_state = None
        if i % 2 == 0:
            new_state = torch.randn(1, 3, 80, 80, device=DEVICE)
        action = torch.randn(1, 1, device=DEVICE)
        reward = torch.randn(1, 1, device=DEVICE)
        test_replay_memory.push(state, action, new_state, reward)
    states_batch, actions_batch, next_states_batch, rewards_batch, non_final_mask = doMakeBatch(test_replay_memory, batch_size)
    assert(non_final_mask.sum() == batch_size//2), "non_final_mask not masking properly."
    print("doMakeBatch test passed.")
    return True

class UnitTestDQN(nn.Module):
    def __init__(self, h, w, num_actions):
        super(UnitTestDQN, self).__init__()
        self.num_actions = num_actions
    def forward(self, x):
        assert(False), "Network should not be queried when epsilon = 1.0." 
        return None

def testSelectAction():
    print("Testing select_action...")
    from scipy.stats import chisquare
    sample_size = 10000
    num_tests = 100
    pass_rate = 0.9
    screen_height = 40
    screen_width = 40
    epsilon = 1.0
    num_actions = 7
    test_results = {True: 0, False: 0}
    significance_level = 0.02
    net = UnitTestDQN(screen_height, screen_width, num_actions).to(DEVICE)
    state = torch.randn(1, 3, 80, 80, device=DEVICE)
    for j in range(num_tests):
        samples = {}
        for i in range(sample_size):
            action, new_epsilon = select_action(state, net, num_actions, epsilon, steps_done = 0, bootstrap_threshold = 2)
            assert(type(action) == torch.Tensor and action.size() == (1,1)), "Action not correct shape."
            assert(new_epsilon == epsilon), "Epsilon should not change during bootstrapping."
            action = action.item()
            if action not in samples:
                samples[action] = 0
            samples[action] = samples[action] + 1
        expected = [sample_size / num_actions] * num_actions
        statistic, pvalue = chisquare(f_obs=list(samples.values()), f_exp=expected)
        test_results[pvalue >= significance_level] += 1
    assert(test_results[True] > pass_rate * num_tests), "Random sample is not from uniform distribution."    
    print("select_action test passed.")
    return True

def testPredictQValues():
    print("Testing doPredictQValues...")
    batch_size = 128
    screen_height = 80
    screen_width = 80
    num_actions = 7
    net = DQN(screen_height, screen_width, num_actions).to(DEVICE)
    states_batch = torch.randn(batch_size, 3, 80, 80, device=DEVICE)
    actions_batch = torch.randint(0, 7, (128, 1), device=DEVICE).long()
    state_action_values = doPredictQValues(net, states_batch, actions_batch)
    assert(type(state_action_values) == torch.Tensor and state_action_values.size() == (128, 1)), "Return value not correct shape."
    print("doPredictQValues test passed.")
    return True

def testPredictNextStateUtilities():
    print("Testing doPredictNextStateUtilities...")
    screen_height = 80
    screen_width = 80
    num_actions = 7
    batch_size = 128
    passed = False
    net = DQN(screen_height, screen_width, num_actions).to(DEVICE)
    # First option to try is that the batch is full sized.

    next_states_batch = torch.ones(batch_size, 3, 80, 80, device=DEVICE)
    non_final_mask = torch.ones(batch_size, dtype=torch.bool, device=DEVICE)
    for i in range(batch_size):
        if i % 2 == 1:
            next_states_batch[i].fill_(0)
            non_final_mask[i] = 0
    next_state_values = doPredictNextStateUtilities(net, next_states_batch, non_final_mask, batch_size)
    assert(type(next_state_values) == torch.Tensor and next_state_values.size() == (batch_size, 1)), "Return value not correct shape (attempt 1)."
    for i in range(batch_size):
        if i % 2 == 1:
            assert(next_state_values[i].sum() == 0), "Element " + str(i) + "is not 0.0 when non_final_mask[i] = 0"
    passed = True

    try:
        next_states_batch = torch.ones(batch_size, 3, 80, 80, device=DEVICE)
        non_final_mask = torch.ones(batch_size, dtype=torch.bool, device=DEVICE)
        for i in range(batch_size):
            if i % 2 == 1:
                next_states_batch[i].fill_(0)
                non_final_mask[i] = 0
        next_state_values = doPredictNextStateUtilities(net, next_states_batch, non_final_mask, batch_size)
        assert(type(next_state_values) == torch.Tensor and next_state_values.size() == (batch_size, 1)), "Return value not correct shape (attempt 1)."
        for i in range(batch_size):
            if i % 2 == 1:
                assert(next_state_values[i].sum() == 0), "Element " + str(i) + "is not 0.0 when non_final_mask[i] = 0"
        passed = True
    except RuntimeError as e:
        print(e)
        print("Will try alternative test.")
    if not passed:
        # Next option is that batch is not full sized.
        try:
            next_states_batch = torch.ones(batch_size-1, 3, 80, 80, device=DEVICE)
            non_final_mask = torch.ones(batch_size, dtype=torch.bool, device=DEVICE)
            non_final_mask[0] = 0
            next_state_values = doPredictNextStateUtilities(net, next_states_batch, non_final_mask, batch_size)
            assert(type(next_state_values) == torch.Tensor and next_state_values.size() == (batch_size, 1)), "Return value not correctd shape (attempt 2)."
            passed = True
        except RuntimeError as e:
            print(e)
            print("No further alternative tests available.")
    if passed:
        print("doPredictNextStateUtilities test passed.")
        return True
    assert(False), "doPredictNextStateUtilities did NOT pass test."

def testComputeExpectedQValues():
    print("Testing doComputeExpectedQValues...")
    batch_size = 128
    gamma = 0.5
    next_state_values = torch.ones(batch_size).unsqueeze(1)
    rewards_batch = torch.ones(batch_size).unsqueeze(1)
    expected_state_action_values = doComputeExpectedQValues(next_state_values, rewards_batch, gamma)
    assert(type(expected_state_action_values) == torch.Tensor and expected_state_action_values.size()[0] == batch_size), "Return value not expected shape."
    for i in range(batch_size):
        assert(expected_state_action_values[i].item() == 1.5), "Element " + str(i) + " doesn't have the correct value."
    print("doComputeExpectedQValues test passed.")
    return True

def testComputeLoss():
    print("Testing doComputeLoss...")
    batch_size = 128
    state_action_values = torch.randn(batch_size, device=DEVICE)
    expected_state_action_values = torch.randn(batch_size, device=DEVICE)
    loss = doComputeLoss(state_action_values, expected_state_action_values)
    assert(type(loss) == torch.Tensor and len(loss.size()) == 0), "Loss not of expected shape."
    print("doComputeLoss test passed.")
    return True


def unit_test():
    testReplayMemory()
    testMakeBatch()
    #testSelectAction()
    testPredictQValues()
    testPredictNextStateUtilities()
    testComputeExpectedQValues()
    testComputeLoss()

## Training loop

In [83]:

### Training loop.
### Each episode is a game that runs until the agent gets the coin or the game times out.
### Train for a given number of episodes.
def train(num_episodes = NUM_EPISODES, load_filename = None, save_filename = None, eval_interval = EVAL_INTERVAL, replay_capacity = REPLAY_CAPACITY, bootstrap_threshold = BOOTSTRAP, epsilon = EPSILON, eval_epsilon = EVAL_EPSILON, gamma = GAMMA, batch_size = BATCH_SIZE, target_update = TARGET_UPDATE, random_seed = RANDOM_SEED, num_levels = NUM_LEVELS, seed = SEED):
    # Set the random seed
    if random_seed is not None:
        random.seed(random_seed)
        torch.manual_seed(random_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(RANDOM_SEED)
    # Set up the environment
    setup_utils.setup_and_load(use_cmd_line_args=False, is_high_res=True, num_levels=num_levels, set_seed=seed)
    env = make('standard', num_envs=1)
    if RENDER_SCREEN and not IN_PYNB:
        env.render()

    # Reset the environment
    env.reset()

    # Get screen size so that we can initialize layers correctly based on shape returned from AI gym. 
    init_screen = get_screen(env)
    _, _, screen_height, screen_width = init_screen.shape
    print("screen size: ", screen_height, screen_width)

    # Are we resuming from an existing model?
    policy_net = None
    if load_filename is not None and os.path.isfile(os.path.join(MODEL_PATH, load_filename)):
        print("Loading model...")
        policy_net = load_model(load_filename)
        policy_net = policy_net.to(DEVICE)
        print("Done loading.")
    else:
        print("Making new model.")
        policy_net = DQN(screen_height, screen_width, env.NUM_ACTIONS).to(DEVICE)
    # Make a copy of the policy network for evaluation purposes
    eval_net = DQN(screen_height, screen_width, env.NUM_ACTIONS).to(DEVICE)
    eval_net.load_state_dict(policy_net.state_dict())
    eval_net.eval()
    # Target network is a snapshot of the policy network that lags behind (for stablity)
    target_net = DQN(screen_height, screen_width, env.NUM_ACTIONS).to(DEVICE)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()
    
    # Instantiate the optimizer
    optimizer = None
    if len(list(policy_net.parameters())) > 0:
        optimizer = initializeOptimizer(policy_net.parameters())
    
    # Instantiate the replay memory
    replay_memory = ReplayMemory(replay_capacity)

    steps_done = 0               # How many steps have been run
    best_eval = float('inf')     # The best model evaluation to date

    ### Do training until episodes complete 
    print("training...")
    i_episode = 0            # The episode number
    
    # Stop when we reach max episodes
    while i_episode < num_episodes:
        print("episode:", i_episode, "epsilon:", epsilon)
        max_reward = 0       # The best reward we've seen this episode
        done = False         # Has the game ended (timed out or got the coin)
        episode_steps = 0    # Number of steps performed in this episode
        # Initialize the environment and state
        env.reset()
        
        # Current screen. There is no last screen because we get velocity on the screen itself.
        state = get_screen(env)

        # Do forever until the loop breaks 
        while not done:
            # Select and perform an action
            action, epsilon = select_action(state, policy_net, env.NUM_ACTIONS, epsilon, steps_done, bootstrap_threshold)
            steps_done = steps_done + 1
            episode_steps = episode_steps + 1
            
            # for debugging
            if RENDER_SCREEN and not IN_PYNB:
                env.render() 

            # Run the action in the environment
            if action is not None: 
                _, reward, done, _ = env.step(np.array([action.item()]))

                # Record if this was the best reward we've seen so far
                max_reward = max(reward, max_reward)
                
                # Turn the reward into a tensor  
                reward = torch.tensor([reward], device=DEVICE)

                # Observe new state
                current_screen = get_screen(env)

                # Did the game end?
                if not done:
                    next_state = current_screen
                else:
                    next_state = None

                # Store the transition in memory
                replay_memory.push(state, action, next_state, reward)

                # Move to the next state
                state = next_state

                # If we are past bootstrapping we should perform one step of the optimization
                if steps_done > bootstrap_threshold:
                  optimize_model(policy_net, target_net if target_update > 0 else policy_net, replay_memory, optimizer, batch_size, gamma)
            else:
                # Do nothing if select_action() is not implemented and returning None
                env.step(np.array([0]))
                
            # If we are done, print some statistics
            if done:
                print("duration:", episode_steps)
                print("max reward:", max_reward)
                status, _ = episode_status(episode_steps, max_reward)
                print("result:", status)
                print("total steps:", steps_done, '\n')

            # Should we update the target network?
            if target_update > 0 and i_episode % target_update == 0:
                target_net.load_state_dict(policy_net.state_dict())
                
        # Should we evaluate?
        if steps_done > bootstrap_threshold and i_episode > 0 and i_episode % eval_interval == 0:
            test_average_duration = 0       # Track the average eval duration
            test_average_max_reward = 0     # Track the average max reward
            # copy all the weights into the evaluation network
            eval_net.load_state_dict(policy_net.state_dict())
            # Evaluate 10 times
            for _ in range(EVAL_COUNT):
                # Call the evaluation function
                test_duration, test_max_reward = evaluate(eval_net, eval_epsilon, env, test_seed=seed)
                status, score = episode_status(test_duration, test_max_reward)
                test_duration = score # Set test_duration to score to factor in death-penalty
                test_average_duration = test_average_duration + test_duration
                test_average_max_reward = test_average_max_reward + test_max_reward
            test_average_duration = test_average_duration / EVAL_COUNT
            test_average_max_reward = test_average_max_reward / EVAL_COUNT
            print("Average duration:", test_average_duration)
            print("Average max reward:", test_average_max_reward)
            # If this is the best window average we've seen, save the model
            if test_average_duration < best_eval:
                best_eval = test_average_duration
                if save_filename is not None:
                    save_model(policy_net, save_filename, i_episode)
            print(' ')
        # Only increment episode number if we are done with bootstrapping
        if steps_done > bootstrap_threshold:
          i_episode = i_episode + 1
    print('Training complete')
    if RENDER_SCREEN and not IN_PYNB:
        env.render()
    env.close()
    return policy_net


## Optimization Function


In [84]:

### Take a DQN and do one forward-backward pass.
### Since this is Q-learning, we will run a forward pass to get Q-values for state-action pairs and then 
### give the true value as the Q-values after the Q-update equation.
def optimize_model(policy_net, target_net, replay_memory, optimizer, batch_size, gamma):
    if len(replay_memory) < batch_size:
        return
    ### step 1: sample from the replay memory. Get BATCH_SIZE transitions
    ### Step 2: Get a list of non-final next states.
    ###         a. Create a mask, a tensor of length BATCH_SIZE where each element i is 1 if 
    ###            batch.next_state[i] is not None and 0 otherwise.
    ###         b. Create a tensor of shape [BATCH_SIZE, color(3), height, width] by concatenating
    ###            all non-final (not None) batch.next_states together.
    ### Step 3: set up batches for state, action, and reward
    ###         a. Create a tensor of shape [BATCH_SIZE, color(3), height, width] holding states
    ###         b. Create a tensor of shape [BATCH_SIZE, 1] holding actions
    ###         c. Create a tensor of shape [BATCH_SIZE, 1] holding rewards
    states_batch, actions_batch, next_states_batch, rewards_batch, non_final_mask = doMakeBatch(replay_memory, batch_size)

    ### Step 4: Get the action values predicted.
    ###         a. Call policy_net(state_batch) to get a tensor of shape [BATCH_SIZE, NUM_ACTIONS] containing Q-values
    ###         b. For each batch, get the Q-value for the corresponding action in action_batch (hint: torch.gather)
    state_action_values = doPredictQValues(policy_net, states_batch, actions_batch)

    ### Step 5: Get the utility values of next_states.
    next_state_values = doPredictNextStateUtilities(target_net, next_states_batch, non_final_mask, batch_size)
    
    ### Step 6: Compute the expected Q values.
    expected_state_action_values = doComputeExpectedQValues(next_state_values, rewards_batch, gamma)

    ### Step 7: Computer Huber loss (smooth L1 loss)
    ###         Compare state action values from step 5 to expected state action values from step 7
    loss = doComputeLoss(state_action_values, expected_state_action_values)
    ### Step 8: Back propagation
    ###         a. Zero out gradients
    ###         b. call loss.backward()
    ###         c. Prevent gradient explosion by clipping gradients between -1 and 1
    ###            (hint: param.grad.data is the gradients. See torch.clamp_() )
    ###         d. Tell the optimizer that another step has occurred: optimizer.step()
    if optimizer is not None:
        optimizer.zero_grad()
        doBackprop(loss, policy_net.parameters())
        optimizer.step()

## Evaluation function

In [85]:

### Evaluate the DQN
### If environment is given, use that. Otherwise make a new environment.
def evaluate(policy_net, epsilon = EVAL_EPSILON, env = None, test_seed = SEED):
    global RUN_NUM
    RUN_NUM = RUN_NUM + 1
    setup_utils.setup_and_load(use_cmd_line_args=False, is_high_res=True, num_levels=NUM_LEVELS, set_seed=test_seed)
    

    # Make an environment if we don't already have one
    if env is None:
        env = make('standard', num_envs=1)
    if RENDER_SCREEN and not IN_PYNB:
        env.render()

    # Reset the environment
    env.reset()

    # Get screen size so that we can initialize layers correctly based on shape
    # returned from AI gym. 
    init_screen = get_screen(env, SCREEN_SAVE)
    _, _, screen_height, screen_width = init_screen.shape

    # Get the network ready for evaluation (turns off some things like dropout if used)
    policy_net.eval()

    # Current screen. There is no last screen
    state = get_screen(env, SCREEN_SAVE)

    steps_done = 0         # Number of steps executed
    max_reward = 0         # Max reward seen
    done = False           # Is the game over?

    print("Evaluating...")
    while not done:
        # Select and perform an action
        action, _ = select_action(state, policy_net, env.NUM_ACTIONS, epsilon, steps_done=0, bootstrap_threshold=0)
        steps_done = steps_done + 1

        if RENDER_SCREEN and not IN_PYNB:
            env.render()          

        # Execute the action
        if action is not None:
            _, reward, done, _ = env.step(np.array([action.item()]))

            # Is this the best reward we've seen?
            max_reward = max(reward, max_reward)

            # Observe new state
            state = get_screen(env, SCREEN_SAVE)
        else:
            # Do nothing if select_action() is not implemented and returning None
            env.step(np.array([0]))

    print("duration:", steps_done)
    print("max reward:", max_reward)
    status, _ = episode_status(steps_done, max_reward)
    print("result:", status, '\n')
    if RENDER_SCREEN and not IN_PYNB:
        env.render()
    return steps_done, max_reward

## DQN

In [117]:
class DQN(nn.Module):

    ### Create all the nodes in the computation graph.
    ### We won't say how to put the nodes together into a computation graph. That is done
    ### automatically when forward() is called.
    def __init__(self, h, w, num_actions):
        super(DQN, self).__init__()
        self.num_actions = num_actions

        # Conv -> Conv -> Conv -> Linear -> Linear

        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1)
        self.relu1 = nn.ReLU()
        self.batchNorm1 = nn.BatchNorm2d(16)
        h_out = h - 5 // 2 * 2
        w_out = w - 5 // 2 * 2

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1)
        self.relu2 = nn.ReLU()
        self.batchNorm2 = nn.BatchNorm2d(32)
        h_out = h_out - 5 // 2 * 2
        w_out = w_out - 5 // 2 * 2

        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.relu3 = nn.ReLU()
        self.batchNorm3 = nn.BatchNorm2d(32)
        h_out = (h_out - 5) // 2 + 1
        w_out = (w_out - 5) // 2 + 1

        linear_in = 32*h_out*w_out
        linear_out = 1024

        self.linear1 = nn.Linear(linear_in, linear_out)
        self.linearRelu = nn.ReLU()
        
        self.linear2 = nn.Linear(linear_out, num_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        q_values = None
        t = self.batchNorm1(self.relu1(self.conv1(x)))
        t = self.batchNorm2(self.relu2(self.conv2(t)))
        t = self.batchNorm3(self.relu3(self.conv3(t)))
        #flatten out image for linear layer
        t = torch.flatten(t, start_dim=1)
        t = self.linearRelu(self.linear1(t))

        q_values = self.linear2(t)
                 
        return q_values

## get_screen

In [118]:
### Take the environment and return a tensor containing screen data as a 3D tensor containing (color, height, width) information.
def get_screen(env, save = False):
    # Returned screen requested by gym is 512x512x3. Transpose it into torch order (Color, Height, Width).
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    _, screen_height, screen_width = screen.shape
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    if save:
      save_screen(screen)
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0).to(DEVICE)

## ReplayMemory

In [119]:
### Store transitions to use to prevent catastrophic forgetting.
### ReplayMemory implements a ring buffer. Items are placed into memory
###    until memory reaches capacity, and then new items start replacing old items
###    at the beginning of the array. 
### Member variables:
###    capacity: (int) number of transitions that can be stored
###    memory: (array) holds transitions (state, action, next_state, reward)
###    position: (int) index of current location in memory to place the next transition.
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    ### Store a transition in memory.
    ### To implement: put new items at the end of the memory array, unless capacity is reached.
    ###    Combine the arguments into a new Transition object.
    ###    If capacity is reached, start overwriting the beginning of the array.
    ###    Use the position index to keep track of where to put the next item. 
    def push(self, state, action, next_state, reward):
        
        new_mem = Transition(state, action, next_state, reward)

        if self.position >= self.capacity:
          self.position = 0
        
        if len(self.memory) < self.capacity:
          self.memory.append(new_mem)

        else:
          self.memory[self.position] = new_mem
        
        self.position += 1
        
    ### Return a batch of transition objects from memory containing batch_size elements.
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    ### This allows one to call len() on a ReplayMemory object. E.g. len(replay_memory)
    def __len__(self):
        return len(self.memory)


## initializeOptimizer

In [120]:
### Choose and instantiate an optimizer
### Input:
### - parameters: the DQN parameters
### Output:
### - the optimizer object
def initializeOptimizer(parameters):
    optimizer = optim.Adam(parameters, lr=0.00005)
    #optimizer = optim.SGD(parameters, lr=0.001)
    return optimizer

## select_action

In [121]:
### Select an action to perform. 
### If a random number [0..1] is greater than epsilon, then query the policy_network,
### otherwise use a random action.
### Inputs:
### - state: a tensor of shape 3 x screen_height x screen_width
### - policy_net: a DQN object
### - num_actions: number of actions available
### - epsilon: float [0..1] indicating whether to choose random or use the network
### - steps_done: number of previously executed steps
### - bootstrap_threshold: number of steps that must be executed before training begins
### This function should return:
### - A tensor of shape 1 x 1 that contains the number of the action to execute
### - The new epsilon value to use next time
def select_action(state, policy_net, num_actions, epsilon, steps_done = 0, bootstrap_threshold = 0):
    global EPSILON_DECAY_NUM

    # state_device = state.to(DEVICE)
    random_num = torch.rand(1)

    if random_num > epsilon:
      # choose policy
      with torch.no_grad():
        action = torch.argmax(policy_net(state)).unsqueeze(0).unsqueeze(0)
    else:
      #random action
      action = torch.randint(low=0, high=num_actions, size=(1,1)).to(DEVICE)
      # action.unsqueeze(0).to(DEVICE)

    if steps_done > bootstrap_threshold:
      # update epsilon
      new_epsilon = 0.1 + (EPSILON - 0.1) * math.exp(-1. * EPSILON_DECAY_NUM / 50000)
      EPSILON_DECAY_NUM += 1
      if new_epsilon < 0.1:
        new_epsilon = 0.1
    
    return action, new_epsilon

## doMakeBatch

In [122]:
### Ask for a batch of experience replays.
### Inputs:
### - replay_memory: A ReplayMemory object
### - batch_size: size of the batch to return
### Outputs:
### - states_batch: a tensor of shape batch_size x 3 x screen_height x screen_width
### - actions_batch: a tensor of shape batch_size x 1 containing action numbers
### - next_states_batch: a tensor containing screens. 
### - rewards_batch: a tensor of shape batch_size x 1 containing reward values.
### - non_final_mask: a tensor of bytes of length batch_size containing a 0 if the state is terminal or 1 otherwise
def doMakeBatch(replay_memory, batch_size):
    batch = replay_memory.sample(batch_size)
    batch = zip(*batch)
    # print("batch should be (state,state,state), (action,action,action), ...")
    # print(batch)
    # print(len(batch))

    states, actions, next, rewards = batch

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, next)), device=DEVICE, dtype=torch.bool)
    
    next_states_batch = torch.cat([s for s in next if s is not None]).to(DEVICE)

    states_batch = torch.cat(states, dim=0).to(DEVICE)
    actions_batch = torch.cat(actions, dim=0).to(DEVICE)
    rewards_batch = torch.cat(rewards, dim=0).to(DEVICE)

    # print("states_batch: ", states_batch.shape)
    # print("actions_batch: ", actions_batch.shape)
    # print("next_states_batch: ", next_states_batch.shape)
    # print("rewards_batch: ", rewards_batch.shape)
    # print("non_final_mask: ", non_final_mask.shape)

    return states_batch, actions_batch, next_states_batch, rewards_batch, non_final_mask

## doPredictQValues

In [123]:
### Ask the policy_net to predict the Q value for a batch of states and a batch of actions.
### Inputs:
### - policy_net: the DQN
### - states_batch: a tensor of shape batch_size x 3 x screen_height x screen_width containing screens
### - actions_batch: a tensor of shape batch_size x 1 containing action numbers
### Output:
### - A tensor of shape batch_size x 1 containing the Q-value predicted by the DQN in the position indicated by the action
def doPredictQValues(policy_net, states_batch, actions_batch):

    state_action_values = policy_net(states_batch).gather(1, actions_batch)

    return state_action_values

## doPredictNextStateUtilities

In [124]:
### Ask the policy_net to predict the utility of a next_state.
### Inputs:
### - policy_net: The DQN
### - next_states_batch: a tensor of shape batch_size x 3 x screen_height x screen_width
### - non_final_mask: a tensor of length batch_size containing 0 for terminal states and 1 for non-terminal states
### - batch_size: the batch size
### Note: Only run non-terminal states through the policy_net
### Output:
### - A tensor of shape batch_size x 1 containing Q-values
def doPredictNextStateUtilities(policy_net, next_states_batch, non_final_mask, batch_size):
    next_state_values = torch.zeros(batch_size, device=DEVICE).float()

    if next_states_batch.shape[0] != torch.count_nonzero(non_final_mask):

      next_states_batch = next_states_batch[non_final_mask].float()

    with torch.no_grad():
      next_state_values[non_final_mask] = torch.argmax(policy_net(next_states_batch), dim=1).float()

    next_state_values = next_state_values.unsqueeze(1)
    
    return next_state_values.detach()

## doComputeExpectedQValues

In [125]:
### Compute the Q-update equation Q(s_t, a_t) = R(s_t+1) + gamma * argmax_a' Q(s_t+1, a')
### Inputs:
### - next_state_values: a tensor of shape batch_size x 1 containing Q values for state s_t+1
### - rewards_batch: a tensor or shape batch_size x 1 containing reward values for state s_t+1
### Output:
### - A tensor of shape batch_size x 1
def doComputeExpectedQValues(next_state_values, rewards_batch, gamma):
    expected_state_action_values = rewards_batch + gamma * next_state_values

    return expected_state_action_values

## doComputeLoss

In [126]:
### Compute the loss
### Inputs:
### - state_action_values: a tensor of shape batch_size x 1 containing Q values
### - expected_state_action_values: a tensor of shape batch_size x 1 containing updated Q values
### Output:
### - A tensor scalar value
def doComputeLoss(state_action_values, expected_state_action_values):

    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values)

    return loss

## doBackprop

In [127]:
### Run backpropagation. Make sure gradients are clipped between -1 and +1.
### Inputs:
### - loss: a tensor scalar
### - parameters: the parameters of the DQN
### There is no output
def doBackprop(loss, parameters):
    loss.backward()
    # print(type(parameters))
    # print(parameters)
    for p in parameters:
      p.grad = torch.clamp(p.grad, -1, 1)


# Unit testing

Run this cell to make sure your functions return valid outputs.

In [ ]:
### RUN UNIT TESTS ON FUNCTIONS
unit_test() 

# Training

Edit the globals as necessary.
The best model produced during training will be saved to ```SAVE_FILENAME``` with a number appended to the end.

###### Train for EASY_LEVEL

In [ ]:
SAVE_FILENAME = 'easy.model'
LOAD_FILENAME = None
TRAIN_SEED = EASY_LEVEL
policy_net = train(num_episodes=NUM_EPISODES, 
                   load_filename=LOAD_FILENAME, 
                   save_filename=SAVE_FILENAME, 
                   eval_interval=EVAL_INTERVAL, 
                   replay_capacity=REPLAY_CAPACITY, 
                   bootstrap_threshold=BOOTSTRAP, 
                   target_update=TARGET_UPDATE,
                   epsilon=EPSILON, 
                   eval_epsilon=EVAL_EPSILON,
                   gamma=GAMMA, 
                   batch_size=BATCH_SIZE,
                   random_seed=RANDOM_SEED,
                   seed=TRAIN_SEED)

In [ ]:
print(EPSILON_DECAY_NUM)

Train for MEDIUM_LEVEL

In [ ]:
SAVE_FILENAME = 'medium.model'
LOAD_FILENAME = None
TRAIN_SEED = MEDIUM_LEVEL
policy_net = train(num_episodes=NUM_EPISODES, 
                   load_filename=LOAD_FILENAME, 
                   save_filename=SAVE_FILENAME, 
                   eval_interval=EVAL_INTERVAL, 
                   replay_capacity=REPLAY_CAPACITY, 
                   bootstrap_threshold=BOOTSTRAP, 
                   target_update=TARGET_UPDATE,
                   epsilon=EPSILON, 
                   eval_epsilon=EVAL_EPSILON,
                   gamma=GAMMA, 
                   batch_size=BATCH_SIZE,
                   random_seed=RANDOM_SEED,
                   seed=TRAIN_SEED)

Train for ONE_MONSTER Level

In [ ]:
SAVE_FILENAME = 'monster.model'
LOAD_FILENAME = None
TRAIN_SEED = ONE_MONSTER
policy_net = train(num_episodes=NUM_EPISODES, 
                   load_filename=LOAD_FILENAME, 
                   save_filename=SAVE_FILENAME, 
                   eval_interval=EVAL_INTERVAL, 
                   replay_capacity=REPLAY_CAPACITY, 
                   bootstrap_threshold=BOOTSTRAP, 
                   target_update=TARGET_UPDATE,
                   epsilon=EPSILON, 
                   eval_epsilon=EVAL_EPSILON,
                   gamma=GAMMA, 
                   batch_size=BATCH_SIZE,
                   random_seed=RANDOM_SEED,
                   seed=TRAIN_SEED)

# Evaluation

This loads the saved model (even if there is a number appended to the end) and evaluates it.

It also produces a dump of screenshots for each run of ```evaluate()``` in ```/content/cache/```.

Evaluate the easy.model

In [ ]:
TEST_FILENAME = 'easy.model'
TEST_SEED = EASY_LEVEL
eval_net = load_model(TEST_FILENAME)

for _ in range(EVAL_COUNT):
  duration, max_reward = evaluate(eval_net, epsilon=EVAL_EPSILON, test_seed=TEST_SEED)

 Evaluate the medium.model

In [ ]:
TEST_FILENAME = 'medium.model'
TEST_SEED = MEDIUM_LEVEL
eval_net = load_model(TEST_FILENAME)

for _ in range(EVAL_COUNT):
  duration, max_reward = evaluate(eval_net, epsilon=EVAL_EPSILON, test_seed=TEST_SEED)

Evaluate the monster.model

In [ ]:
TEST_FILENAME = 'monster.model'
TEST_SEED = ONE_MONSTER
eval_net = load_model(TEST_FILENAME)

for _ in range(EVAL_COUNT):
  duration, max_reward = evaluate(eval_net, epsilon=EVAL_EPSILON, test_seed=TEST_SEED)

Easy Model

In [ ]:
TEST_FILENAME = 'easy.model'
TEST_SEED = EASY_LEVEL
eval_net = load_model(TEST_FILENAME)
best_easy_duration = None
best_easy_grade = 0.0

try:
  for i in range(10):
    print("EASY LEVEL TRY", i)
    easy_grade = 0.0
    total_duration = 0.0
    for j in range(10):
      duration, max_reward = evaluate(eval_net, epsilon = EVAL_EPSILON, test_seed = TEST_SEED)
      status, score = episode_status(duration, max_reward)
      total_duration = total_duration + score
    average_duration = total_duration / 10.0
    if average_duration < 150:
      easy_grade = easy_grade + 3
    if average_duration < 100:
      easy_grade = easy_grade + 1
    if average_duration < 50:
      easy_grade = easy_grade + 1
    if easy_grade > best_easy_grade:
      best_easy_grade = easy_grade
    if best_easy_duration is None or average_duration < best_easy_duration:
      best_easy_duration = average_duration
    print("EASY SCORE", i, easy_grade, average_duration)
    if easy_grade >= 5.0:
      break
except Exception as e: 
  print("EASY GRADING BROKE")

print("BEST EASY SCORE", best_easy_grade, best_easy_duration)

Medium Model

In [ ]:
TEST_FILENAME = 'medium.model'
TEST_SEED = MEDIUM_LEVEL
eval_net = load_model(TEST_FILENAME)
best_medium_duration = None
best_medium_grade = 0.0

try:
  for i in range(10):
    print("MEDIUM LEVEL TRY", i)
    medium_grade = 0.0
    total_duration = 0.0
    for j in range(10):
      duration, max_reward = evaluate(eval_net, epsilon = EVAL_EPSILON, test_seed = TEST_SEED)
      status, score = episode_status(duration, max_reward)
      total_duration = total_duration + score
    average_duration = total_duration / 10.0
    if average_duration < 150:
      medium_grade = medium_grade + 1
    if medium_grade > best_medium_grade:
      best_medium_grade = medium_grade
    if best_medium_duration is None or average_duration < best_medium_duration:
      best_medium_duration = average_duration
    print("MEDIUM SCORE", i, medium_grade, average_duration)
    if medium_grade >= 1.0:
      break
except Exception as e: 
  print("MEDIUM GRADING BROKE")

print("BEST MEDIUM SCORE", best_medium_grade, best_medium_duration)

Monster (Hard) Model

In [ ]:
TEST_FILENAME = 'monster.model'
TEST_SEED = ONE_MONSTER
eval_net = load_model(TEST_FILENAME)
best_monster_duration = None
best_monster_grade = 0.0

try:
  for i in range(10):
    print("MONSTER LEVEL TRY", i)
    monster_grade = 0.0
    total_duration = 0.0
    for j in range(10):
      duration, max_reward = evaluate(eval_net, epsilon = EVAL_EPSILON, test_seed = TEST_SEED)
      status, score = episode_status(duration, max_reward)
      total_duration = total_duration + score
    average_duration = total_duration / 10.0
    if average_duration < 300:
      monster_grade = monster_grade + 1
    if monster_grade > best_monster_grade:
      best_monster_grade = monster_grade
    if best_monster_duration is None or average_duration < best_monster_duration:
      best_monster_duration = average_duration
    print("MONSTER SCORE", i, monster_grade, average_duration)
    if monster_grade >= 1.0:
      break
except Exception as e: 
  print("MONSTER GRADING BROKE")

print("BEST MONSTER SCORE", best_monster_grade, best_monster_duration)

# Download The Model

If you want to download the model to a computer without NVIDIA GPU and CUDA support, you can use the cell below. It takes the model indicated, converts it to run on CPU, and saves it. You can then open the file browser to the left and download the model file.<br/> <br/><b>Change the model name in load_model argument below to download medium and monster models respectively.</b>

In [ ]:
### USE THIS TO COVERT A MODEL FROM CUDA TO CPU
### DOWNLOAD USING THE FILES MENU TO THE LEFT

### CHANGE THE MODEL NAME BELOW FOR MEDIUM AND MONSTER MODELS
net = load_model('easy.model')
net = net.to('cpu')
torch.save(net, os.path.join(MODEL_PATH, 'saved_cpu.model'))

# Show Evaluation Movies

The following cells will render an animation in the notebook. It is flickery but you should get an idea of what your agent was duing. 

In [ ]:
# This seems to work but is really slow and flickery

eval_run_number = 1
show_movie(eval_run_number)

Alternatively, the following makes an animated gif. The animation is faster and doesn't flicker once loaded. Unfortunately, colab loads animated gifs really slow.
Better to download and view on one's own machine (though this is cumbersome).

In [ ]:
eval_run_number = 1
make_anim(eval_run_number)